In [2]:
from lxml import etree
import glob

In [3]:
from pathlib import Path
from tqdm import tqdm

In [4]:
import pandas as pd
import numpy as np
import os,sys

In [5]:
import re
from PIL import Image

### 1.Data Loader.

In [66]:
root = Path('/nas/chenyi/datasets_nas/dada_det/xml/wardrobe500k/wardrobe20k_20220630/xml_20220629/')

In [67]:
xml_list = glob.glob(str(root /'*/*/*.xml'))
# xml_list += glob.glob(str(root /'*/*/*.xml'))
# xml_list += glob.glob(str(root /'*/*/*/*.xml'))
len(xml_list)

10596

In [68]:
# 核查xml是否有重复
xml_df = pd.DataFrame()
xml_df['path'] = xml_list
xml_df['stem'] = xml_df['path'].apply(lambda x: Path(x).stem)
len(xml_df)

10596

### 接口

In [69]:
def voc2yolo():
    center_x = (xmin + (xmax-xmin)/2)/width
    center_y = (ymin + (ymax-ymin)/2)/height
    bdb_w = (xmax-xmin)/width
    bdb_h = (ymax-ymin)/height

In [70]:
def parse_xml(p):
    trees = etree.parse(p)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    bboxes = []
    for obj in trees.findall('object'):
        bx = []
        bx.append(p)
        bx.append(int(width))
        bx.append(int(height))
        bdb = obj.find('bndbox')
        bx.append(obj.find('name').text)
        bx.append(int(bdb.find('xmin').text))
        bx.append(int(bdb.find('ymin').text))
        bx.append(int(bdb.find('xmax').text))
        bx.append(int(bdb.find('ymax').text))
        bboxes.append(bx)
    return bboxes

In [100]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}
model_label_map8 = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装_短款
    3: 5,  # 裤子_短款
    4: 6,  # 裙子
    5: 7,  # 连体装
    6: 4,  # 上装_长款
    7: 5,  # 裤子_长款
}

In [103]:
chi_label2id8 = {
    0: '鞋子',
    1: '包包',
    2: '上装_短款',
    3: '裤子_短款',
    4: '裙子',
    5: '连体装',
    6: '上装_长款',
    7: '裤子_长款',
}

In [81]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [104]:
id2label_chi8 = get_list2dict(chi_label2id8.values(), chi_label2id8.keys())

### 解析XML

In [46]:
!ls /nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia

det_nodetagain2  nodet	nodet_kfolder  nodet_watoo470k.tar  SAMPLE_DEL


In [47]:
!du -h /nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/

12G	/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_watoo470k.tar


In [ ]:
# !cp /nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_watoo470k.tar \
#     /nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_watoo470k.tar

In [82]:
# !ls /nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/

#### csv保存和读取

In [29]:
root

PosixPath('/nas/chenyi/datasets_nas/dada_det/xml/wardrobe500k/wardrobe20k_20220630/xml_20220629')

In [27]:
!ls $root

wuzhangjing1_20220626  wuzhangjing3_20220629  wuzhanjing2_20220628


In [28]:
!mkdir -p $root/info

In [108]:
# bboxes_df.to_csv(root /'info/xml_20220629.csv', index=False)

#### 解析XML

In [71]:
bboxes_list = []
for p in tqdm(xml_list):
    row = parse_xml(p)
    bboxes_list += row
    # break

100%|███████████████████████| 10596/10596 [00:07<00:00, 1427.01it/s]


In [72]:
bboxes_df = pd.DataFrame(bboxes_list)
name_stand = ['xml_path', 'width', 'height', 'labelimg', 'bx1', 'bx2','bx3', 'bx4']
bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
                          bboxes_df.columns[1]:name_stand[1],
                          bboxes_df.columns[2]:name_stand[2],
                          bboxes_df.columns[3]:name_stand[3],
                          bboxes_df.columns[4]:name_stand[4],
                          bboxes_df.columns[5]:name_stand[5],
                          bboxes_df.columns[6]:name_stand[6],
                          bboxes_df.columns[7]:name_stand[7],
                         },
                          inplace=True)

In [73]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4
0,/nas/chenyi/datasets_nas/dada_det/xml/wardrobe...,1080,1554,裙子,6,478,1080,1129


In [74]:
bboxes_df.head(1).values

array([['/nas/chenyi/datasets_nas/dada_det/xml/wardrobe500k/wardrobe20k_20220630/xml_20220629/wuzhangjing1_20220626/skirt/2017022320122485968.xml',
        1080, 1554, '裙子', 6, 478, 1080, 1129]], dtype=object)

In [75]:
bboxes_df.value_counts('labelimg')

labelimg
裙子       4357
裤子_长款    2618
上装_短款    1962
裤子_短款    1250
鞋子        484
包包        242
上装_长款     117
连体装        34
dtype: int64

In [76]:
bboxes_df['stem'] = bboxes_df['xml_path'].apply(lambda x: Path(x).stem)

#### 获取对应图片信息

In [49]:
!ls /nas/chenyi/datasets_nas/dada_det/data/wardrobe500k/wardrobe500k_hard/

@eaDir	__MACOSX  wartoo470k_hard_nodet_  wartoo470k_hard_nodet_.zip


In [77]:
img_list = glob.glob('/nas/chenyi/datasets_nas/dada_det/data/wardrobe500k/wardrobe500k_hard/wartoo500k_hard20220629/nodet_kfolder/*/*.jpg')
img_df = pd.DataFrame()
img_df['path'] = img_list
len(img_df)

20553

In [78]:
img_df['stem'] = img_df['path'].apply(lambda x: Path(x).stem)

In [79]:
# 去重
img_df.drop_duplicates('stem', inplace=True)
len(bboxes_df), len(img_df)

(11064, 20553)

##### 获取bbox对应的图片

In [80]:
# xml是否都有对应的图片
cross_df = img_df[img_df['stem'].isin(bboxes_df['stem'])]
len(cross_df)

10593

In [83]:
cross_df.head(1)

,path,stem
0,/nas/chenyi/datasets_nas/dada_det/data/wardrob...,2017020200393708851


In [84]:
stem2path = get_list2dict(cross_df['stem'].values, cross_df['path'].values)

In [87]:
bboxes_df['img_sp'] = bboxes_df['stem'].map(stem2path)

##### 获取图片的宽和高

In [88]:
w_list = []
h_list = []
for ind,row in tqdm(bboxes_df.iterrows()):
    img_sp = row['img_sp']
    img = Image.open(img_sp)
    w, h = img.size
    w_list.append(w)
    h_list.append(h)
    # break

11064it [00:32, 336.85it/s]


In [91]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp
0,/nas/chenyi/datasets_nas/dada_det/xml/wardrobe...,1080,1554,裙子,6,478,1080,1129,2017022320122485968,/nas/chenyi/datasets_nas/dada_det/data/wardrob...


In [90]:
df['width'] = w_list
df['height'] = h_list

#### label处理

In [107]:
bboxes_df.value_counts('labelimg')

labelimg
裙子       4357
裤子_长款    2618
上装_短款    1962
裤子_短款    1250
鞋子        484
包包        242
上装_长款     117
连体装        34
dtype: int64

In [106]:
bboxes_df['label86'] = bboxes_df['labelimg'].map(chi_label2id8)

##### 统计人员的bbox

In [22]:
# df['man'] = df[int(df.columns[0])].apply(lambda x: re.findall('wholebody0609_xml/.*?_(.*?)/', x)[0])
# df.value_counts('man')

### 数据展示

In [24]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/dada_det

/home/chenyi/workspace/dataset/data/dada_det
├── hard_sample_0610
├── __MACOSX
├── shoe_bag11k_may
├── shoeBag5k0515
└── wholebody0601_all

5 directories


In [25]:
ROOT = Path('/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/')

#### 处理打标标签

##### 构建中文标签和业务id映射

In [45]:
# counter = df.value_counts(df.columns[3])
# name_list = [item[0] for item in list(counter.index)]
# name_df = pd.DataFrame()
# name_df['name'] = name_list

In [121]:
import json
with open('../../../data/dict/name_chi2l1.json', 'r') as f:
    name_chi2l1 = json.load(f)

In [122]:
df['label6'] = df[df.columns[3]].map(name_chi2l1)

In [123]:
df.value_counts('label6')

label6
4    27
7    19
6    8 
8    4 
5    2 
dtype: int64

In [115]:
# 增加多出的中文标签映射
# name_chi2l1['皮衣外套'] = 4

In [116]:
import json
with open('../../../data/dict/name_chi2l1.json', 'w') as f:
    json.dump(name_chi2l1, f)

In [124]:
model_label_map

{0: 10, 1: 8, 2: 4, 3: 5, 4: 6, 5: 7}

In [126]:
l1_map_model = get_list2dict(model_label_map.values(), model_label_map.keys())
df['label_model'] = df['label6'].map(l1_map_model)

##### 存储目标index

In [ ]:
# df = pd.read_csv('../data/wh.csv')

In [ ]:
!ls /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
!mkdir -p /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed')

In [ ]:
df.head(2).values

In [ ]:
df['img_tp'] = df['img_sp'].apply(lambda x:'{0}/{1}/{2}'.format(draw_root, Path(x).parts[-2], Path(x).name))

In [ ]:
sys.path.append('../../../')
from utils.plots.plots import draw_bbox
import cv2
from PIL import Image

In [ ]:
# error_p_list = []
# for ind, row in tqdm(df.iterrows()):
#     row_info = row.loc[df.columns[:8]].values
#     w,h = row_info[1], row_info[2]
#     name = row.loc[['label_model']].values[0]
#     bbox = list(row_info[4:8])
#     bbox.insert(0,name)
#     tp = Path(row.loc[['img_tp']].values[0])
#     if not tp.parent.is_dir():
#         tp.parent.mkdir()
#     rp = tp if tp.is_file() else row.loc[['img_sp']].values[0]
#     img = cv2.imread(str(rp))
#     img = draw_bbox(img, bbox)
#     try:
#         cv2.imwrite(str(tp), img)
#     except:
#         error_p_list.append(tp)
#     # break

In [ ]:
error_p_list

In [ ]:
error_df = pd.DataFrame()
error_df['path'] = error_p_list
error_df['stem'] = error_df['path'].apply(lambda x: Path(x).stem)

In [ ]:
df.head()

In [ ]:
df = df[~df['stem'].isin(error_df['stem'])]

In [ ]:
# df.to_csv('../data/wholebody5k0610/wholebody5k_july.csv', index=False)

### Display Html

In [ ]:
sys.path.append('../../../')
from utils.display.html import *
from IPython.display import HTML
from io import BytesIO

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed/')

In [ ]:
img_list = glob.glob(str(draw_root/'*/*.jpg'))
len(img_list), img_list[-1]

In [ ]:
# img_list = glob.glob(draw_root + '*.jpg')
df_drawed = pd.DataFrame()
df_drawed['path'] = img_list[-100:]

In [ ]:
size = 500
df_drawed['res'] = df_drawed['path'].map(lambda x: get_thumbnail(x, size))

In [ ]:
f = df_drawed[['res', 'path']].to_html(formatters={'res': image_formatter}, escape=False)

In [ ]:
HTML(f)

### 解析json文件

In [ ]:
json_list = glob.glob('/home/chenyi/workspace/dataset/haowei34k_labelImg/xml_label/*/*.json')

In [ ]:
json_df = pd.DataFrame()
json_df['spath'] = json_list

In [ ]:
xml_dir = Path('/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/')

In [ ]:
json_df['tpath'] = json_df['spath'].apply(lambda x:
                                         '{0}/{1}'.format(xml_dir, re.findall(r'/[0-9]{1,3}/(.*?.json)',x)[0]))

In [ ]:
sys.path.append('/home/chenyi/workspace/myPyModule99/data_format/json_to_xml')
from json_to_xml.main import *

In [ ]:
root_json_dir = xml_dir
root_save_xml_dir = '/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/xml'

In [ ]:
import json
for json_filename in tqdm(os.listdir(root_json_dir)):
    json_path = os.path.join(root_json_dir, json_filename)
    json_data = json.load(open(json_path))
    break

In [ ]:
json_df['path'] = glob.glob(str(root_json_dir/'*.json'))

In [ ]:
json_df['fname'] =  json_df['path'].apply(lambda x: re.findall(r'/json2xml/(.*?).json',x)[0])

In [ ]:
json_labels = {}
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    json_labels[fname] = ''
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        json_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
            classes_name[name], xmin, ymin, xmax, ymax)

In [ ]:
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

In [ ]:
json_bboxes = pd.DataFrame()

In [ ]:
json_bboxes['fname'] = fname_list
json_bboxes['label'] = labels_list
json_bboxes['xmin'] = xmin_list
json_bboxes['ymin'] = ymin_list
json_bboxes['xmax'] = xmax_list
json_bboxes['ymax'] = ymax_list

In [ ]:
json_bboxes.head()